In [1]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

# library(msigdbr)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




Here I perform enrichment analysis to find modules enriched for cell type markers. These modules will later be used to correlate to exon PSI to find cell type-specific exons.

In [3]:
# Get DE genes frome each cell type: pooled analysis
pooled_res <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [ ]:
pval_threshold <- .05/length(pooled_res)

ctype_genes <- lapply(pooled_res, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > 2)
    df[mask, 1]
})
names(ctype_genes) <- names(pooled_res) 

In [15]:
network_dir <- "mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules"

In [17]:
# Traverse networks to get cell type enrichments for each module

networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
networks <- networks[lengths(lapply(networks, list.files)) > 0]

network_enrichments <- lapply(seq_along(networks), function(i) {
    
    kme_file <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
    kme <- fread(file.path(networks[i], kme_file), data.table=FALSE)
    mod_col <- grep("PosFDR", colnames(kme))
    mod_genes <- tapply(kme$Gene, kme[,mod_col], list)
    
    if (length(mod_genes) > 0) {
        all_genes <- kme$Gene

        # For each module calculate enrichment for top kME genes
        mod_enrichments <- lapply(mod_genes, function(mod) {
            lapply(unlist(lapply(ctype_genes, function(set) {
                fisher_test(set, mod, all=all_genes)
            })), c)
        })
        
        network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
        mod_enrichments_df <- reshape2::melt(mod_enrichments)
        colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
        
        me_file <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
        
        data.frame(
            Network=network_id,
            kME_path=file.path(networks[i], kme_file),
            ME_path=file.path(networks[i], me_file),
            mod_enrichments_df
        )
    }
    
})
enrichments_df <- do.call(rbind, network_enrichments)

In [ ]:
enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

top_mods_df <- all_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    slice_min(Qval) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
head(top_mods_df)